In [1]:
import time
import torch
from torch import nn,optim
import torch.nn.functional as F
import sys

In [2]:
sys.path.append(r'F:\study\ml\python_packages')
import d2l

loading customized package : d2l 


In [3]:
class Residual(nn.Module):
    def __init__(self,in_channels,out_channels,use_1x1conv=False,stride=1):
        super().__init__()
        self.conv1=nn.Conv2d(in_channels,out_channels,kernel_size=3,padding=1,stride=stride)
        self.conv2=nn.Conv2d(out_channels,out_channels,kernel_size=3,padding=1)
        if use_1x1conv:
            self.conv3=nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=stride)
        else:
            self.conv3=None
        self.bn1=nn.BatchNorm2d(out_channels)
        self.bn2=nn.BatchNorm2d(out_channels)
    
    def forward(self,X):
        Y=F.relu(self.bn1(self.conv1(X)))
        Y=self.bn2(self.conv2(Y))
        if self.conv3:
            X=self.conv3(X)
        return F.relu(Y+X)  

In [4]:
class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return F.avg_pool2d(x,kernel_size=x.size()[2:])

In [5]:
blk=Residual(3,3)
X=torch.rand((4,3,6,6))
blk(X).shape

torch.Size([4, 3, 6, 6])

In [6]:
blk=Residual(3,6,use_1x1conv=True,stride=2)
blk(X).shape

torch.Size([4, 6, 3, 3])

In [16]:
net=nn.Sequential(
    nn.Conv2d(1,64,kernel_size=7,stride=2,padding=3),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3,stride=2,padding=1) 
)

In [8]:
X2=torch.rand(4,1,96,96)
for cnet in net.children():
    X2=cnet(X2)
    print('net shape : ',X2.shape)

net shape :  torch.Size([4, 64, 48, 48])
net shape :  torch.Size([4, 64, 48, 48])
net shape :  torch.Size([4, 64, 48, 48])
net shape :  torch.Size([4, 64, 24, 24])


In [9]:
def resnet_block(in_channels,out_channels,num_residuals,first_block=False):
    if first_block:
        assert in_channels == out_channels
    blk=[]
    for i in range(num_residuals):
        if i ==0 and not  first_block:
            blk.append(Residual(in_channels,out_channels,use_1x1conv=True,stride=2))
        else:
            blk.append(Residual(out_channels,out_channels))
        return nn.Sequential(*blk)

观察每个block的形状
- net.add_module("resnet_block1", resnet_block(64, 64, 2,first_block=True))
- net.add_module("resnet_block2", resnet_block(64, 128, 2))
- net.add_module("resnet_block3", resnet_block(128, 256, 2))
- net.add_module("resnet_block4", resnet_block(256, 512, 2))

In [17]:
net.add_module("resnet_block1", resnet_block(64, 64, 2,first_block=True))

In [18]:
net

Sequential(
  (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (resnet_block1): Sequential(
    (0): Residual(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
)

In [19]:
net.add_module("resnet_block2", resnet_block(64, 128, 2))

In [20]:
net

Sequential(
  (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (resnet_block1): Sequential(
    (0): Residual(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (resnet_block2): Sequential(
    (0): Residual(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2))
      (bn1): BatchNorm2d(128, eps=1e-05, mom

In [21]:
net.add_module("resnet_block3", resnet_block(128, 256, 2))
net.add_module("resnet_block4", resnet_block(256, 512, 2))

In [22]:
net.add_module("global_avg_pool", GlobalAvgPool2d()) #
net.add_module("fc", nn.Sequential(d2l.FlattenLayer(),
nn.Linear(512, 10)))

In [23]:
net

Sequential(
  (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (resnet_block1): Sequential(
    (0): Residual(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (resnet_block2): Sequential(
    (0): Residual(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2))
      (bn1): BatchNorm2d(128, eps=1e-05, mom

In [24]:
X=torch.rand((1,1,224,224))
for name ,layer in net.named_children():
    X=layer(X)
    print(name,'output shape : \t',X.shape)

0 output shape : 	 torch.Size([1, 64, 112, 112])
1 output shape : 	 torch.Size([1, 64, 112, 112])
2 output shape : 	 torch.Size([1, 64, 112, 112])
3 output shape : 	 torch.Size([1, 64, 56, 56])
resnet_block1 output shape : 	 torch.Size([1, 64, 56, 56])
resnet_block2 output shape : 	 torch.Size([1, 128, 28, 28])
resnet_block3 output shape : 	 torch.Size([1, 256, 14, 14])
resnet_block4 output shape : 	 torch.Size([1, 512, 7, 7])
global_avg_pool output shape : 	 torch.Size([1, 512, 1, 1])
fc output shape : 	 torch.Size([1, 10])


注意观察:在人为增加通道数的同时,利用1*1卷积的stride,减小了高和宽

In [27]:
batch_size=256
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size,resize=96)
lr,num_epochs=0.001,5
optimizer=torch.optim.Adam(net.parameters(),lr=lr)
d2l.train_ch5(net,train_iter,test_iter,batch_size,optimizer,num_epochs)

epoch 1,loss 0.3803,train acc 0.864,test acc 0.895,time 1590.0 sec
epoch 2,loss 0.1161,train acc 0.915,test acc 0.892,time 1565.8 sec
epoch 3,loss 0.0651,train acc 0.928,test acc 0.918,time 1532.8 sec
epoch 4,loss 0.0405,train acc 0.941,test acc 0.916,time 1513.6 sec
epoch 5,loss 0.0268,train acc 0.951,test acc 0.915,time 1758.6 sec
